# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [139]:
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [6]:
df['posse_de_veiculo'] = df['posse_de_veiculo'].astype(int)
df['posse_de_imovel'] = df['posse_de_imovel'].astype(int)

In [11]:
df

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,0,1,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,1,1,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,1,1,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,0,1,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,1,0,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,16006,F,0,1,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,14996,2016-03-01,3722,F,0,1,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,14997,2016-03-01,6194,F,1,1,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,14998,2016-03-01,4922,M,1,0,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  int32  
 5   posse_de_imovel        15000 non-null  int32  
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [15]:
df.isnull().value_counts()

Unnamed: 0  data_ref  id_cliente  sexo   posse_de_veiculo  posse_de_imovel  qtd_filhos  tipo_renda  educacao  estado_civil  tipo_residencia  idade  tempo_emprego  qt_pessoas_residencia  renda
False       False     False       False  False             False            False       False       False     False         False            False  False          False                  False    12427
                                                                                                                                                    True           False                  False     2573
Name: count, dtype: int64

In [17]:
df.dropna(inplace=True)

In [19]:
df.isnull().any()

Unnamed: 0               False
data_ref                 False
id_cliente               False
sexo                     False
posse_de_veiculo         False
posse_de_imovel          False
qtd_filhos               False
tipo_renda               False
educacao                 False
estado_civil             False
tipo_residencia          False
idade                    False
tempo_emprego            False
qt_pessoas_residencia    False
renda                    False
dtype: bool

In [21]:
y = df['renda']
X = df.drop(columns=['renda','data_ref','sexo','tipo_renda', 'educacao', 'estado_civil',  'tipo_residencia'])

In [23]:
X.columns

Index(['Unnamed: 0', 'id_cliente', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia'],
      dtype='object')

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 47)

In [27]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

In [29]:
for alpha in alphas:
    rid = Ridge(alpha=alpha)
    rid.fit(X_train, y_train)
    y_pred = rid.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2

In [31]:
results_mse, results_r2

({0: 50338908.15152994,
  0.001: 50338907.80507385,
  0.005: 50338906.41925984,
  0.01: 50338904.68701548,
  0.05: 50338890.829986826,
  0.1: 50338873.5110163},
 {0: 0.17099570246058515,
  0.001: 0.1709957081661836,
  0.005: 0.17099573098840615,
  0.01: 0.17099575951580315,
  0.05: 0.1709959877197259,
  0.1: 0.17099627293649988})

In [33]:
min(results_mse.values()), max(results_r2.values())

(50338873.5110163, 0.17099627293649988)

O modelo com menor mse e maior R² foi o modelo que apresenta alpha igual 0.1.

In [35]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

In [37]:
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2

C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.330e+11, tolerance: 7.933e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

In [39]:
results_mse, results_r2

({0: 50338908.15152993,
  0.001: 50338905.32074289,
  0.005: 50338893.99825748,
  0.01: 50338879.846549,
  0.05: 50338766.694163375,
  0.1: 50338625.40262821},
 {0: 0.17099570246058537,
  0.001: 0.1709957490792886,
  0.005: 0.17099593554318715,
  0.01: 0.17099616860003297,
  0.05: 0.1709980320455744,
  0.1: 0.17100035889957577})

In [41]:
min(results_mse.values()), max(results_r2.values())

(50338625.40262821, 0.17100035889957577)

O modelo com menor mse e maior R² foi o modelo que apresenta alpha igual 0.1.

Ridge -> mse = 50338873.5110163, R² = 0.17099627293649988 
Lasso -> mse = 50338625.40262821, R² = 0.17100035889957577 


Verificando os dados acima podemos concluir que o modelo utilizando Lasso foi um pouco melhor, mas a diferença não foi significativa.

In [ ]:
######################################################

In [43]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adiciona {best_feature} com p-value de {best_pval}')

        # Backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Exclui {worst_feature} com p-value de {worst_pval}')
        if not changed:
            break
    return included

result = stepwise_selection(X_train, y_train)

print('Variáveis escolhidas:')
print(result)


Adiciona tempo_emprego com p-value de 0.0
Adiciona posse_de_veiculo com p-value de 1.5145557629592038e-34
Adiciona qt_pessoas_residencia com p-value de 0.0010877484943821849
Variáveis escolhidas:
['tempo_emprego', 'posse_de_veiculo', 'qt_pessoas_residencia']


In [45]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

In [47]:
for alpha in alphas:
    rid = Ridge(alpha=alpha)
    rid.fit(X_train[result], y_train)
    y_pred = rid.predict(X_test[result])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2

In [49]:
results_mse, results_r2

({0: 50340574.58180139,
  0.001: 50340574.3015607,
  0.005: 50340573.18060456,
  0.01: 50340571.779424384,
  0.05: 50340560.57058261,
  0.1: 50340546.561029285},
 {0: 0.17096825892024414,
  0.001: 0.17096826353537675,
  0.005: 0.17096828199579817,
  0.01: 0.17096830507107785,
  0.05: 0.1709684896634408,
  0.1: 0.17096872037920996})

In [51]:
min(results_mse.values()), max(results_r2.values())

(50340546.561029285, 0.17096872037920996)

In [53]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

In [55]:
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train[result], y_train)
    y_pred = lasso.predict(X_test[result])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2

C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.334e+11, tolerance: 7.933e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

In [57]:
results_mse, results_r2

({0: 50340574.58180139,
  0.001: 50340573.156326674,
  0.005: 50340567.45831239,
  0.01: 50340560.31324326,
  0.05: 50340503.81046892,
  0.1: 50340432.620100826},
 {0: 0.17096825892024414,
  0.001: 0.17096828239561757,
  0.005: 0.1709683762331381,
  0.01: 0.17096849390142355,
  0.05: 0.1709694244151052,
  0.1: 0.1709705968108377})

In [59]:
min(results_mse.values()), max(results_r2.values())

(50340432.620100826, 0.1709705968108377)

MSE 50338873.5110163 R² = 0.17099627293649988

Ridge -> mse = 50338873.5110163, R² = 0.17099627293649988
Lasso -> mse = 50338625.40262821, R² = 0.17100035889957577
Stepwise_Ridge -> mse = 50340546.561029285, R² = 0.17096872037920996
Stepwise_Lasso -> mse = 50340432.620100826, R² = 0.1709705968108377 

In [ ]:
O melhor resultado foi do modelo que utilizou Lasso.

In [ ]:
Os dados acima mostram que todos os modelos ficaram muito próximos, mas o modelo utilizando Stepwise e Lasso ficou mais simples. Desta forma, ele é o escolhido.

In [ ]:
##############################################################################

In [61]:
X['tempo_emprego'] = np.log(X['tempo_emprego'])
X['qt_pessoas_residencia'] = np.log(X['qt_pessoas_residencia'])

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 47)

In [67]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

In [69]:
for alpha in alphas:
    rid = Ridge(alpha=alpha)
    rid.fit(X_train[result], y_train)
    y_pred = rid.predict(X_test[result])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2
min(results_mse.values()), max(results_r2.values())

(54469645.50073606, 0.10296881947396319)

In [71]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train[result], y_train)
    y_pred = lasso.predict(X_test[result])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2
min(results_mse.values()), max(results_r2.values())

C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.570e+11, tolerance: 7.933e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

(54469464.019309364, 0.10297180819358731)

Aplicando LOG não melhorou

In [75]:
y = df['renda']
X = df.drop(columns=['renda','data_ref','sexo','tipo_renda', 'educacao', 'estado_civil',  'tipo_residencia'])
X['tempo_emprego'] = np.power(X['tempo_emprego'],2)
X['qt_pessoas_residencia'] = np.power(X['qt_pessoas_residencia'],2)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 47)

In [81]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}
for alpha in alphas:
    rid = Ridge(alpha=alpha)
    rid.fit(X_train[result], y_train)
    y_pred = rid.predict(X_test[result])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2
min(results_mse.values()), max(results_r2.values())

(50944028.939559385, 0.16103029494128052)

In [83]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train[result], y_train)
    y_pred = lasso.predict(X_test[result])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2
min(results_mse.values()), max(results_r2.values())

C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.340e+11, tolerance: 7.933e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

(50943928.67412624, 0.16103194615855998)

Elevando ao quadrado também não ficou legal.

In [119]:
y = df['renda']
X = df.drop(columns=['renda','data_ref','sexo','tipo_renda', 'educacao', 'estado_civil',  'tipo_residencia'])
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 47)

In [123]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}
for alpha in alphas:
    rid = Ridge(alpha=alpha)
    rid.fit(X_train[result], y_train)
    y_pred_log = rid.predict(X_test[result])
    
    # Inverter a transformação logarítmica
    y_pred = np.exp(y_pred_log)
    y_test_original = np.exp(y_test)  # Inverter a transformação logarítmica de y_test também

    
    mse = mean_squared_error(y_test_original, y_pred)
    r2 = r2_score(y_test_original, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2
min(results_mse.values()), max(results_r2.values())

(54293953.69231928, 0.10586219300088895)

In [121]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_mse = {}
results_r2 = {}

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train[result], y_train)
    y_pred_log = lasso.predict(X_test[result])

    # Inverter a transformação logarítmica
    y_pred = np.exp(y_pred_log)
    y_test_original = np.exp(y_test)  # Inverter a transformação logarítmica de y_test também

    
    mse = mean_squared_error(y_test_original, y_pred)
    r2 = r2_score(y_test_original, y_pred)
    results_mse[alpha] = mse
    results_r2[alpha] = r2
min(results_mse.values()), max(results_r2.values())

C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.991e+03, tolerance: 7.550e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

(54293953.692319304, 0.10586219300088862)

Também não melhorou o modelo aplicando o Log apenas em y

In [ ]:
Nenhuma das modificações melhorou o modelo

In [135]:
y = df['renda']
X = df.drop(columns=['renda','data_ref','sexo','tipo_renda', 'educacao', 'estado_civil',  'tipo_residencia'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 47)

In [141]:
# Ajustar a Árvore de Regressão
tree = DecisionTreeRegressor(random_state=47)
tree.fit(X_train[result], y_train)
y_pred= tree.predict(X_test[result])

mse_tree = mean_squared_error(y_test, y_pred)
r2_tree = r2_score(y_test, y_pred)

print(mse_tree, r2_tree)

43130662.59063825 0.2897044064656602


In [ ]:
Melhorou consideravelmente o resultado fazendo o ajuste pela árvore.